Laplacian Mesh Representation notebook
========

In [1]:
import jax.numpy as jnp
from jax import jacfwd
import numpy as np
import meshplot as mp

## Utility functions

Firstly, we need to define some helper functions for
1. Loading in a mesh (we will only work with the .obj format)
2. Creating adjacency matrices
3. Pre-computing areas and normals of triangles

In [2]:
def objloader(folder, fname):
    V = []
    F = []
    with open(folder + fname, 'r') as f:
        lines = f.readlines()
        for l in lines:
            token = l.split(' ')[0]
            if token == 'v':
                V.append(jnp.array([float(v) for v in l.split(' ')[1:]]))
            if token == 'f':
                F.append(jnp.array([int(f.split('/')[0]) - 1 for f in l.split(' ')[1:]]))
    V = jnp.array(V)
    F = jnp.array(F, dtype=jnp.int32)
    return V, F
    
def compute_vertex_neighbourhood_matrix(V, F):
    N = np.zeros((V.shape[0], V.shape[0]), dtype=np.int)
    for f in F:
        N[f[0], [f[1], f[2]]] = 1
        N[f[1], [f[0], f[2]]] = 1
        N[f[2], [f[0], f[1]]] = 1
    return jnp.array(N)

def compute_face_neighbourhood_matrix(V, F, E):
    N = np.zeros((V.shape[0], F.shape[0]), dtype=np.int)
    for fi, f in enumerate(F):
        N[f, fi] = 1
    return jnp.array(N)

def compute_edge_connectivity(V, F):
    E = np.zeros((V.shape[0], V.shape[0]), dtype=np.int)
    E[F[:, 0], F[:, 1]] = 1
    E[F[:, 1], F[:, 2]] = 1
    E[F[:, 2], F[:, 0]] = 1
    # Length of edges
    E_lens = np.zeros((V.shape[0], V.shape[0]))
    E_lens[F[:, 0], F[:, 1]] = jnp.linalg.norm(V[F[:, 1]] - V[F[:, 0]], axis=1)
    E_lens[F[:, 1], F[:, 2]] = jnp.linalg.norm(V[F[:, 2]] - V[F[:, 1]], axis=1)
    E_lens[F[:, 2], F[:, 0]] = jnp.linalg.norm(V[F[:, 0]] - V[F[:, 2]], axis=1)
    return jnp.array(E), jnp.array(E_lens)

def compute_normals_and_areas(V, F):
    Ns = jnp.cross(V[F[:, 1]] - V[F[:, 0]], V[F[:,2]] - V[F[:, 0]])
    A = jnp.linalg.norm(Ns, axis=1) * 0.5
    Ns = Ns / jnp.linalg.norm(Ns, axis=1).reshape(Ns.shape[0], 1)
    return jnp.array(Ns), jnp.array(A)

def compute_voronoi_cell_areas(V, F, NF):
    VA = np.zeros((V.shape[0]))
    for vi, v in enumerate(V):
        for f in F[NF[vi] == 1]:
            vs = V[f]
            # Get mid-point
            mid_point = jnp.mean(vs, axis=0)
            # Get edge mid-points
            vj, vk = [fi for fi in f if fi != vi]
            meij = (V[vj] - v) / 2.
            meik = (V[vk] - v) / 2.
            meim = (mid_point - v)
            # Compute the two triangle areas
            VA[vi] += jnp.linalg.norm(jnp.cross(meij, meim))
            VA[vi] += jnp.linalg.norm(jnp.cross(meik, meim))
    return VA

def compute_mesh_dual_points(V, F, E):
    ac = V[F[:, 2]] - V[F[:, 0]]
    ab = V[F[:, 1]] - V[F[:, 0]]
    abXac = jnp.cross(ab, ac)
    
    aclen2 = jnp.linalg.norm(ac, axis=1)**2
    ablen2 = jnp.linalg.norm(ab, axis=1)**2
    abXaclen2 = jnp.linalg.norm(abXac, axis=1)**2
    
    centre = jnp.multiply(
                (jnp.multiply(jnp.cross(abXac, ab).T, aclen2).T +\
                 jnp.multiply(jnp.cross(ac, abXac).T, ablen2).T).T,
                1. / (2. * abXaclen2)).T
    
    return jnp.mean(V[F], axis=1)

def compute_vertex_face_participation_matrix(V, F):
    VF = np.zeros((V.shape[0], F.shape[0]), dtype=np.int)
    for vi, _ in enumerate(V):
        VF[vi, np.where(F == vi)[0]] = 1
    return VF

def compute_dual_edges(V, F, D, VF, E):
    DE = np.zeros((E.shape[0], E.shape[1],3 ))
    DEl = np.zeros((E.shape[0], E.shape[1]))
    DA = np.zeros((V.shape[0]))
    
    Eindices = np.where(E == 1)
    
    E0 = Eindices[0] # "From" vertices
    E1 = Eindices[1] # "To" vertices
    
    F_range = jnp.array([jnp.arange(F.shape[0])])
    F_pairs = F_range[(VF[E0] & VF[E1]) == 1] # Get the face pairs sharing each edge
    F_pairs = F_pairs.reshape((F_pairs.shape[0] // 2, 2))
    
    DE[E0, E1] = D[F_pairs[:, 1]] - D[F_pairs[:, 0]]
    for i in range(DA.shape[0]):
        DA[i] = 0.5 * jnp.sum(jnp.linalg.norm(jnp.cross(V[i, :] - (-1)*DE[i, np.where(E[i, :] == 1)], DE[i, np.where(E[i, :] == 1)]), axis=2))
    DEl = jnp.linalg.norm(DE, axis=2)
    return DEl, DE, DA

def precompute_mesh_attributes(folder, fname):
    import time
    now = time.time()
    V, F = objloader(folder, fname)
    print("Loading in mesh:", time.time() - now, "s")
    now = time.time()
    E, El = compute_edge_connectivity(V, F)
    print("edge connectivity + lens:", time.time() - now, "s")
    now = time.time()
    NV = compute_vertex_neighbourhood_matrix(V, F)
    print("vertex neighbourhood connectivity:", time.time() - now, "s")
    now = time.time()
    NF = compute_face_neighbourhood_matrix(V, F, E)
    print("face neighbourhood connectivity:", time.time() - now, "s")
    now = time.time()
    N, A = compute_normals_and_areas(V, F)
    print("normals + area:", time.time() - now, "s")
    now = time.time()
    D = compute_mesh_dual_points(V, F, E)
    print("dual points:", time.time() - now, "s")
    now = time.time()
    VF = compute_vertex_face_participation_matrix(V, F)
    print("vertex - face adjacency:", time.time() - now, "s")
    now = time.time()
    DEl, DE, DA = compute_dual_edges(V, F, D, VF, E)
    print("dual edge computation:", time.time() - now, "s")
    return V, F, NF, E, NV, N, A, VF, D, El, DEl, DE, DA

### Loading in a Mesh

In [3]:
DATA = './meshes/'
meshfile = 'stanford_bunny_original.obj'

V, F, NF, E, NV, N, A, VF, D, El, DEl, DE, DA = precompute_mesh_attributes(DATA, meshfile)

d = mp.plot(np.array(V), np.array(F), return_plot=True, shading={'wireframe': True})
d.add_points(np.array(D[0:2, :]), shading={"point_size": 0.03, "point_color": "blue"})

/home/max/anaconda3/lib/python3.7/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


Loading in mesh: 3.022491216659546 s
edge connectivity + lens: 0.25345468521118164 s
vertex neighbourhood connectivity: 0.08222579956054688 s
face neighbourhood connectivity: 0.06087613105773926 s
normals + area: 0.10664010047912598 s
dual points: 0.22161102294921875 s
vertex - face adjacency: 0.5459408760070801 s
dual edge computation: 15.677211999893188 s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016800…

1

## Computing the Laplacian operator (Crane)

In [2], the Laplacian operator $\Delta u = f$ is given by,
$$
(\Delta u)_i = \frac{1}{2}\sum\limits_j (\cot \alpha_j + \cot \beta_j)(u_i - u_j)
$$

In other words, we need to construct a matrix operator, which transforms each vertex of a mesh into its cotan weighted Laplacian representation.

From [2] we know that we can express the Laplacian, $\Delta = \nabla \cdot \nabla$ using discrete exterior calculus as,
$$
\star d \star du = f
$$
where $\star$ denotes the Hodge star operator.

Starting from a 0-form $u$ (i.e. a number $u_i$ at each vertex), we can compute the discrete exterior derivative, $du$, which corresponds to integrating the derivative along each edge:
$$
(du)_{ij} = \int\limits_{e_{ij}} du
$$
Now, using Stokes theorem we see that,
$$
\int\limits_{e_{ij}} du = 
\int\limits_{\partial e_{ij}} u =
u_j - u_i
$$
which means that the discrete derivative of $u_i$ is simply the edge going from vertex $i$ to vertex $j$.

The Hodge star operator converts the circulation along the edge $e_{ij}$ into the flux through the corresponding dual edge.
For a triangle mesh, the dual of an edge, $e^\star_{ij}$ corresponds to the
edge going from between the midpoints of the triangles sharing edge $e_{ij}$.
To compute $(\star du)_{ij}$, we therefore need to take the total circulation along the primal edge, given by $u_j - u_i$, dividing by the length of the primal edge ($|e_{ij}|$), and multiplying by the length of the dual edge,
$$
(\star du)_{ij} = \frac{|e^\star_{ij}|}{|e_{ij}|}(u_j - u_i)
$$

Taking the derivative of this quantity corresponds to integrating over the whole dual cell (i.e. w.r.t. all primal and dual edges of a vertex $i$),
$$
(d \star du)_{i} = \int\limits_{C_i} d \star du =
\int\limits_{\partial C_i} \star du =
\sum\limits_j \frac{|e^\star_{ij}|}{|e_{ij}|}(u_j - u_i)
$$
where we again applied Stokes theorem, and $C_i$ corresponds to the area of the entire dual cell.

The final Hodge star corresponds to dividing by the total area of the cell, which leaves us with a final formulation of the Laplacian as,
$$
(\star d \star du)_i = \frac{1}{C_i}\sum\limits_j \frac{|e^\star_{ij}|}{|e_{ij}|}(u_j - u_i) = f\Rightarrow
$$
$$
(\Delta u)_i = \frac{1}{C_i}\sum\limits_j \frac{|e^\star_{ij}|}{|e_{ij}|}(u_j - u_i) = f
$$

Usually, this is transformed s.t.,
$$
\star d \star du = f \Rightarrow
d \star du = \star f
$$
since $d \star du$ is then symmetrical.
Thus, we end up with the final relation given by,
$$
(\Delta u)_i = \sum\limits_j \frac{|e^\star_{ij}|}{|e_{ij}|}(u_j - u_i) = C_i f
$$

In [14]:
# Initialize the Laplacian
L = np.zeros((V.shape[0], V.shape[0]))
# Initialize the weights (i.e. the length fo the dual edge divided by the length of the primal edge)
L[np.where(E==1)] = jnp.multiply(DEl[DEl != 0], 1./El[El != 0])
# Initialize the diagonal as the sum of all these weights
L[np.diag_indices_from(L)] = -np.sum(L, axis=1)
# Divide all entries by the area of the i'th element
# TODO: Implement dual cell area properly
# currently, it produces the wrong results

## Example 1) Mean Curvature Flow

In [22]:
# Forward Euler
f0x = V[:, 0]
f0y = V[:, 1]
f0z = V[:, 2]

h = 250.75

fhx = -f0x - h * jnp.matmul(L, f0x)
fhy = -f0y - h * jnp.matmul(L, f0y)
fhz = -f0z - h * jnp.matmul(L, f0z)

fhforward = jnp.vstack([fhx, fhy, fhz]).T * -1

# Backwards Euler
fhx = jnp.linalg.solve(jnp.eye(L.shape[0], dtype=jnp.float32) - h * L, f0x)
fhy = jnp.linalg.solve(jnp.eye(L.shape[0], dtype=jnp.float32) - h * L, f0y)
fhz = jnp.linalg.solve(jnp.eye(L.shape[0], dtype=jnp.float32) - h * L, f0z)

fhbackward = jnp.vstack([fhx, fhy, fhz]).T

d = mp.subplot(np.array(V), np.array(F), c=np.array([1.0, 0.0, 0.0]),  s=[1, 2, 0])
#mp.subplot(np.array(fhforward), np.array(F), c=np.array([1.0, 0.0, 0]), s=[3, 1, 1], data=d)
mp.subplot(np.array(fhbackward), np.array(F), c=np.array([1.0, 0.0, 1.0]), s=[1, 2, 1], data=d)

## Computing the Laplacian operator (Sorkine)

The Laplacian is given by,
$$
    L\mathbf{x} = \delta^{(x)}
$$

where $L$ is the Laplacian operator defined as,

$$
    L = I - D^{(-1)}A
$$

Given matrix $D$, for which

$$
    (D)_{ij} =
    \begin{cases}
    d_i, & \text{ if } i = j\\
    0, & \text{ otherwise }
    \end{cases}
$$

where $d_i = \Omega_i$ (i.e. the size of the voronoi cell of vertex $i$) and $A$ as,

$$
(A)_{ij} =
\begin{cases}
1, & \text{ if } (i, j) \in E\\
0, & \text{ otherwise }
\end{cases}
$$

(See [1])

As we have already computed $D$ and $A$, we simply have:

In [6]:
D = jnp.diag(jnp.sum(E, axis=1))
L = jnp.eye(V.shape[0], dtype=jnp.float32) - jnp.dot(jnp.linalg.inv(D), E)

We also have the symmetric matrix $L_s$ given by,
$$
L_s = D - A
$$

In [7]:
L_s = D - E

## Computing the cotan weights

All which is left for us, is to compute the weights $\delta_i$ given by,
$$
\delta_i = \frac{1}{\Omega_i}\sum\limits_{j \in N(i)} \frac{1}{2} (\cot \alpha_{j} + \cot \beta_{j})(v_i - v_j)
$$
This formulation is also refered to as the *cotan formula*.
Notice here, that there is a difference in the sign between the cotan formulation in [1] and [2].

From [2], we know that we can compute the cotan formula as,
$$
\nabla_{v_i} A = \frac{1}{2}\sum\limits_j (\cot \alpha_j +\cot \beta_j)(v_i - v_i) 
$$
(Here, we flipped the sign of the edge difference from $(v_j - v_i) \to (v_i - v_j)$, as is done in [1]).
Thus, if we just compute the area of all triangles in the neighbourhood of $v_i$, we can compute the cotan weights
by taking the derivative using JAX.

In [8]:
# Unfortunately, we have to recompute the area in order to take the gradient.
# However, this is rather quick since we already know the neighbourhood of each vertex.
def compute_area(V, F, NF):
    Ns = jnp.cross(V[F[:, 1]] - V[F[:, 0]], V[F[:,2]] - V[F[:, 0]])
    A = jnp.linalg.norm(Ns, axis=1) * 0.5
    return jnp.matmul(NF, A)

# We can then simply compute the deltas as:
deltas = jacfwd(compute_area)(V, F, NF).diagonal().T
deltas = jnp.multiply(deltas.T, 1. / compute_area(V, F, NF)).T

## Solving for absolute coordinates

Now that we have expressed our mesh as,
$$
L\mathbf{v} = \delta
$$
we're almost ready to use the Laplacian.
In the applications we're concerned with, we will be changing $\delta$, which means that we're interested in the
relation,
$$
\mathbf{v} = L^{-1}\delta
$$
which means we should be able to restore our mesh using the $\delta$ we computed before altering them.

In [9]:
Vreconstructed = jnp.matmul(jnp.linalg.inv(L), deltas)

However, these are not the correct values!

In [10]:
jnp.linalg.norm(V - Vreconstructed)

DeviceArray(2.4771748e+09, dtype=float32)

In [11]:
d = mp.subplot(np.array(Vreconstructed), np.array(F), c=np.array([1.0, 0.0, 0.0]),  s=[1, 2, 0])
mp.subplot(np.array(V), np.array(F), c=np.array([0.0, 0.0, 1.0]), s=[1, 2, 1], data=d)

This error is caused by the need to "anchor" down some of our vertices, as they will otherwise be translation invariant.
We can without loss of generality add $m$ constraints $C$, s.t.
$$
v_j = c_j,  j \in C
$$

Our system of equations then becomes,
$$
\begin{pmatrix}
L\\
\hline
\omega I_{m \times m} | 0
\end{pmatrix} =
\begin{pmatrix}
\delta\\
c_{1:m}
\end{pmatrix}
$$

Defining,
$$ \tilde{L} =
\begin{pmatrix}
L\\
\hline
\omega I_{m \times m} | 0
\end{pmatrix}
$$
we can then solve this system of equations as
$$
\mathbf{v} = (\tilde{L}^T\tilde{L})^{-1}\tilde{L}^T\begin{pmatrix}
\delta, &
c_{1:m}
\end{pmatrix}^T
$$

to reconstruct the mesh.

In [12]:
print(V.shape[0])
n = L.shape[1] # Number of columns in L
m = 100 # Num constraints
omega = 1.0
Imxm = jnp.eye(m, dtype=jnp.float32)
# Choose the m first vertices as being fixed
c = V[:m]
L_tilde = np.zeros((L.shape[0] + m, L.shape[1]))
L_tilde[:L.shape[0], :L.shape[1]] = L
L_tilde[L.shape[0]:, :m] = Imxm
L_tilde = jnp.array(L_tilde)

rhs = np.zeros((deltas.shape[0] + m, 3))
rhs[:deltas.shape[0], :] = deltas
rhs[deltas.shape[0]:, :] = omega * c
rhs = jnp.array(rhs)

Vreconstructed = jnp.dot(jnp.dot(jnp.linalg.inv(jnp.dot(L_tilde.T, L_tilde)), L_tilde.T), rhs)
d = mp.subplot(np.array(Vreconstructed), np.array(F), c=np.array([1.0, 0.0, 0.0]),  s=[1, 2, 0])
mp.subplot(np.array(V), np.array(F), c=np.array([0.0, 0.0, 1.0]), s=[1, 2, 1], data=d)
print("Residual:", jnp.linalg.norm(Vreconstructed - V))

2503


Residual: 45433.207


References
-------

[1] **Differential Representations for Mesh Processing**, *Sorkine, Olga*, 2006, *Computer Graphics Forum*, Vol. 25

[2] **Discrete Differential Geometry: An Applied Introduction**, *Crane, Keenan*, 2020, Notices of the AMS, Communication